In [ ]:
%load_ext autoreload
# %load_ext pyinstrument
%autoreload 2

In [1]:
import numpy as np
from tqdm import tqdm

import xarray
import glob
from pathlib import Path


import sys
sys.path.append('../')
from toffy import streak_detection as sd
from ark.utils import load_utils

In [2]:
fov_dirs = list(map(Path, glob.glob("../toffy/data/streak/*/")))


streak_data = {}

for fov_dir in tqdm(fov_dirs):
    streak_data[fov_dir.stem] = sd.streak_correction(
        fov_dir, image_name="Noodle", mask_data=True
    )
    streak_data[fov_dir.stem].save_data(all_data=True)


100%|██████████| 3/3 [00:13<00:00,  4.49s/it]


In [3]:
fov_data = load_utils.load_imgs_from_dir(data_dir=fov_dir, dtype=np.int32)


In [ ]:
fov_data

In [ ]:
for data in fov_data:
    name = str(data.fovs.values)
    print(type(name))